In [113]:
import os
import warnings



warnings.filterwarnings("ignore")


### Document Loader

In [ ]:
### Read the list of PDFs in the dir

from langchain_community.document_loaders import PyMuPDFLoader
import os

pdfs = []
for root, dirs, files in os.walk("rag-dataset\Labeling RAG"):
    # print(root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

In [115]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    temp = loader.load()
    docs.extend(temp)

    #print(temp)
    #break

### Document Chunking

In [116]:
from langchain_text_splitters import  RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)

### Document Vector Embedding

In [117]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

In [118]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

In [119]:
vector = embeddings.embed_query("Hello World")

In [120]:
len(vector)
index = faiss.IndexFlatL2(len(vector))
index.ntotal, index.d

(0, 768)

In [121]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [122]:
vector_store.index.ntotal, vector_store.index.d

(0, 768)

In [123]:
ids = vector_store.add_documents(documents=chunks)

In [124]:
### Retreival
#question = "What ist the scope of the 21CFR Part 11"
""" question = "what is should I consider bevor implementation of Signed electronic records "
docs = vector_store.search(query=question, k=5, search_type="similarity") """

' question = "what is should I consider bevor implementation of Signed electronic records "\ndocs = vector_store.search(query=question, k=5, search_type="similarity") '

In [125]:
#docs

In [126]:
#db_name = "swt_validation"
#db_name = "datenschutz_gesetze"
#db_name = "qualitaetsmanagement"
#db_name = "zulassung_von_MED"
db_name = "labeling"
vector_store.save_local(db_name)